# Deep Reinforcement Learning
dt.: *Bestärkendes Lernen*

Für wen?

Schon einfache Computerspiele sind als Umgebung zu komplex um mit einfacher Klassifizierung zu einer brauchbaren Lösung zu kommen. Ursachen:

* *supervised*: Jede Trainingsinstanz hat ein Label.
  * Sogar in der einfachen Umgebung eines Computerspieles bräuchte man sehr viele Instanzen -> Screenshots für jede Situation/Spielzustand
  * Spielzüge, die sich erst später im Spiel auswirken bleiben unberücksichtigt.


* *Reinforcement Learning*: zwischen *supervised* und *unsupervised* *learning*. Manche Zustände bringen *rewards*. Diese Rewards können als (spärliche und zeitversetze) Labels betrachtet werden. Über *Reinforcement Learning* werden Wege dorthin durch Bewertung von Zuständen, die für sich genommen noch keinen Erfolg bringen, gelernt.

### Schwierigkeiten:
* **credit assignment problem**: Welche der Züge waren für das Bekommen der Belohnung verantwortlich?
* **explore-exploit dilemma**: soll eine einmal gefundene, funktionierende Strategie beibehalten werden (*exploitation*), oder ist es zielführend eine nach einer besseren zu suchen (*exploration*)?

# Markov Decision Process (MDP)

Eine übliche Art den Sachverhalt zu beschreiben ist die Betrachtung als *Markov Decision Process*.

* Aktionen (*actions*) führen zu einem neuen Zustand (*state*) und können die Umgebung verändern.
* *actions* führen manchmal zu Belohnungen durch das *environment* oder einen Interpreter des Zustands

<img src="images/actor_state_environment.png" alt="[Die Reinforcement Learning Komponenten: Actor State Environment" style="width: 300px;"/>
Die Reinforcement Learning Komponenten: Actor State Environment<br>Source: selfmade

* Die Regeln zur Wahl einer Aktion werden unter *policy* zusammengefasst.
* Die Umgebung ist nicht zwingend deterministisch, dh. ausgehend vom gleichen Zustand führt eine Aktion nicht immer zum gleichen Folgezustand (Zufallselemente -> stochastisches Verhalten)
* Eine *Episode* ist die Abfolge von *states*, *actions*, *rewards* bis zum Spielende (finaler Zustand $s_n$): $s_0,a_0,r_1, s_1,a_1,r_2, ...s_{n-1},a_{n-1},r_n,s_n$

<img src="images/markov_decision_process.png" alt="Markov Decision Process Graph" style="width: 300px;"/>
Markov Decision Process Graph<br>Source: Wikipedia

* Wichtige Eigenschaft von *MDP*: Der nächste Zustand $s_{i+1}$ (bzw. die Wahrscheinlichkeit dafür) hängt ausschließlich vom derzeitigen Zustand $s$ und der gewählten Aktion $a$ ab, nicht aber von den Zuständen davor.

# Zukünftige Belohnungen diskontieren


## Experience Replay

Aufeinanderfolgende Situationen  (*transitions*) sind oft recht ähnlich und führen mitunter zu lokalen Minima.

Um das zu verhindern ist die wichtigste Technik ***experience replay***: Während des Spiels werden die Erfahrungen ($s, a, r, s'$) gespeichert (*replay memory*). Für das Trainieren des Netzwerkes werden anschließlich zufällig Werte aus dem *replay memory* genommen.

Die Erfahrungen können auch von menschlichen Spielern gesammelt werden.
Geht in Richtung *supervised learning*.

## Deep Q-Learning Algorithmus

bis zu Spielende wiederholen:
* im Zustand $s$ eine Aktion $a$ wählen
  * Mit Wahrscheinlichkeit $\epsilon$ eine zufällige Aktion wählen
  * andernfalls $a = argmax_{a'} Q(s, a')$
* Aktion $a$ durchführen
* *reward* $r$ und neuer Zustand $s'$ aus dem Environment (oder einem Interpreter) erfassen
* Erfahrung ($s, a, r, s'$) im *replay memory* $D$ speichern
* zufälligen Zustandwechsel aus dem *replay memory* wählen